# Module 5 — Radial Velocities (radvel) with Real Data

This notebook shows how to:

1. Simulate an RV dataset and fit a Keplerian using `radvel` (or fall back to `lmfit` if `radvel` is not installed).
2. Instructions and code to load a real RV table (e.g., from the Exoplanet Archive RADIAL dataset or a published CSV) and run the same fit.

**Note:** For a fully-real run on 51 Peg b, download a published RV table (link provided below) and set `rv_filename` to point to the file.

In [ ]:

# Install required packages (run once)
# !pip install radvel pandas numpy matplotlib astropy emcee corner tqdm lmfit
print('If radvel is not installed, uncomment the pip install line above to install it.')


## Part A — Simulate RV data and fit with radvel (or lmfit fallback)

In [ ]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt

# Synthetic RV dataset (single planet, small eccentricity)
np.random.seed(42)
P_true = 4.23077
K_true = 56.0
e_true = 0.01
w_true = 150.0 * np.pi/180.0
t0_true = 0.5
gamma_true = 0.0

# observation times
t = np.sort(np.random.uniform(0, 40, 60))

# Keplerian helper (solve for E via Newton iteration)
def kepler_E(M, e, tol=1e-10):
    E = M.copy()
    for _ in range(50):
        f = E - e*np.sin(E) - M
        df = 1 - e*np.cos(E)
        E -= f/df
    return E

M = 2*np.pi*(t - t0_true)/P_true
E = kepler_E(M, e_true)
nu = 2*np.arctan2(np.sqrt(1+e_true)*np.sin(E/2), np.sqrt(1-e_true)*np.cos(E/2))
rv_true = gamma_true + K_true*(np.cos(nu + w_true) + e_true*np.cos(w_true))

rv_obs = rv_true + np.random.normal(0, 5.0, size=rv_true.size)  # 5 m/s noise
rv_err = np.full_like(rv_obs, 5.0)

plt.errorbar(t, rv_obs, yerr=rv_err, fmt='o', label='synthetic')
plt.plot(t, rv_true, 'k--', label='true model')
plt.xlabel('Time [days]'); plt.ylabel('RV [m/s]'); plt.legend(); plt.show()

# Save to DataFrame for potential radvel ingestion
rv_df = pd.DataFrame({'time': t, 'rv': rv_obs, 'err': rv_err, 'tel': ['inst']*len(t)})
rv_df.head()


### Try fitting with `radvel` (if installed). If not installed, the notebook will use `lmfit` as a fallback.

In [ ]:

# Attempt to fit with radvel; if missing, fallback to lmfit least-squares
try:
    import radvel
    HAS_RADVEL = True
except Exception as e:
    print('radvel not available; will use lmfit fallback. Error:', e)
    HAS_RADVEL = False

if HAS_RADVEL:
    print('radvel is available — you can convert rv_df into a radvel dataset and run a full radvel pipeline.')
    print('Radvel requires defining parameter objects and a ' + "'rtb' file or parameter dictionary for full functionality.")
else:
    # Simple LM fit using lmfit (if installed) or scipy.optimize
    try:
        from lmfit import Parameters, minimize
        def model_kepler(params, t):
            P = params['P'].value
            K = params['K'].value
            e = params['e'].value
            w = params['w'].value
            t0 = params['t0'].value
            M = 2*np.pi*(t - t0)/P
            E = kepler_E(M, e)
            nu = 2*np.arctan2(np.sqrt(1+e)*np.sin(E/2), np.sqrt(1-e)*np.cos(E/2))
            return K*(np.cos(nu + w) + e*np.cos(w))
        def resid(params, t, data, err):
            return (data - model_kepler(params, t)) / err
        params = Parameters()
        params.add('P', value=P_true, min=0.1, max=100.0)
        params.add('K', value=K_true, min=0.1, max=500.0)
        params.add('e', value=e_true, min=0.0, max=0.9)
        params.add('w', value=w_true, min=0.0, max=2*np.pi)
        params.add('t0', value=t0_true, min=0.0, max=P_true)
        out = minimize(resid, params, args=(t, rv_obs, rv_err))
        print('LMFIT fit results:')
        for pn in out.params:
            print(pn, out.params[pn].value, '+/-', out.params[pn].stderr)
        rv_fit = model_kepler(out.params, t)
        plt.errorbar(t, rv_obs, yerr=rv_err, fmt='o', label='data')
        plt.plot(t, rv_fit, 'r-', label='fit')
        plt.legend(); plt.show()
    except Exception as e:
        from scipy.optimize import least_squares
        def residuals(p):
            P, K, e, w, t0 = p
            M = 2*np.pi*(t - t0)/P
            E = kepler_E(M, e)
            nu = 2*np.arctan2(np.sqrt(1+e)*np.sin(E/2), np.sqrt(1-e)*np.cos(E/2))
            model = K*(np.cos(nu + w) + e*np.cos(w))
            return (rv_obs - model)/rv_err
        p0 = [P_true, K_true, e_true, w_true, t0_true]
        res = least_squares(residuals, p0)
        print('Least-squares fit:', res.x)
        P_fit, K_fit, e_fit, w_fit, t0_fit = res.x
        M = 2*np.pi*(t - t0_fit)/P_fit
        E = kepler_E(M, e_fit)
        nu = 2*np.arctan2(np.sqrt(1+e_fit)*np.sin(E/2), np.sqrt(1-e_fit)*np.cos(E/2))
        model_fit = K_fit*(np.cos(nu + w_fit) + e_fit*np.cos(w_fit))
        plt.errorbar(t, rv_obs, yerr=rv_err, fmt='o')
        plt.plot(t, model_fit, 'r-')
        plt.show()


## Part B — Load real RV file and fit

To fit real RV data (e.g., 51 Peg), download a published RV table (e.g., from the RADIAL bulk or a paper), save it as a CSV with columns: `time, rv, err, tel` and set `rv_filename` below.

In [ ]:

# USER: set path to your RV CSV file here
rv_filename = '/path/to/51peg_rv.csv'  # replace with your real file

import os
if not os.path.exists(rv_filename):
    print('RV file not found. Please download a radial velocity file for your target and set rv_filename.')
else:
    real_df = pd.read_csv(rv_filename)
    print(real_df.head())
    # Convert to arrays
    t_real = real_df['time'].values
    rv_real = real_df['rv'].values
    err_real = real_df.get('err', np.full(len(rv_real), 5.0)).values
    tel_real = real_df.get('tel', ['inst']*len(rv_real)).values
    # Next: call radvel pipeline or the fitting code above to fit the real data
    print('Loaded real RV with', len(t_real), 'points. You can now run the fitting code (Part A) with these arrays.')


### Where to find real RV files for 51 Peg

- The RADIAL bulk from NASA Exoplanet Archive contains many literature RV tables (download and search for '51 Peg').
- Paper supplements (e.g., Mayor & Queloz 1995, Naef et al. 2004, Martins et al. 2015) often include tables in the article or online.
- HARPS public RV catalog (ESO) includes measurements for many stars including 51 Peg (see HARPS DR release).

If you want, I can attempt to download a known 51 Peg RV table and embed it in this notebook — tell me if you want me to try that now.